In [1]:
#libraries used
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [26]:
#import of the data set
# Making a list of missing value types
missing_values = ["            "," ","  ","UNKNOWN","#VALUE!"]
data = pd.read_csv('GSAF5.csv', na_values = missing_values, encoding='latin-1')

In [27]:
#check the number of null values per column
null_cols = data.isnull().sum()
null_cols[null_cols > 0]
#Time have 3213 null values -> don't keep it because if this information is often missing, we won't be able to get a trend on the attack hour
#Species have 2934 null values -> keep it because we might still want to know what kind of shark attack
#Age have 2681 null values -> don't keep it as it is not an interresting information if often missing
#I wanted to remove Name column as I don't find this information useful but sometimes it contains sex and other information

Country                     43
Area                       402
Location                   497
Activity                   528
Name                       200
Sex                        567
Age                       2683
Injury                      27
Fatal (Y/N)                114
Time                      3215
Species                   2940
Investigator or Source      15
href formula                 1
href                         3
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

In [28]:
#Drop not interresting columns
to_drop = ['Year','Age','Time','href','Unnamed: 22','Unnamed: 23']
data.drop(columns=to_drop,inplace=True)

In [29]:
#function to clean the date from Case number attribute
def clean_Date(e):

    if re.search("[0-9]{4}.[0-9]{2}.[0-9]{2}",e):
        splitted_date = re.split("\D",e)
        if splitted_date[1] == "00":
            splitted_date[1] = "01"
        if splitted_date[2] == "00":
            splitted_date[2] = "01"
        cleaned_date = datetime(int(splitted_date[0]),int(splitted_date[1]),int(splitted_date[2]))
    else:
        cleaned_date = np.nan
    
    return cleaned_date

In [30]:
"""Is it useful?
#function to change empty value by "Not specified"
def empty_values(e):
    if re.search("^$",e):
        e = "Not specified"
    return e"""

'Is it useful?\n#function to change empty value by "Not specified"\ndef empty_values(e):\n    if re.search("^$",e):\n        e = "Not specified"\n    return e'

In [31]:
#function to remove space at the beginning and at the end of my string
def remove_spaces(e):
    if re.search("^ ",e):
        e = re.sub("^ +","",e)
    if re.search(" $",e):
        e = re.sub(" +$","",e)
    return e

In [32]:
"""Does not work
def sex_cleaning(e):
    if ((e != 'F') or (e != 'M')):
        e = "Not specified" #retourne que du not specified
    return e"""

'Does not work\ndef sex_cleaning(e):\n    if ((e != \'F\') or (e != \'M\')):\n        e = "Not specified" #retourne que du not specified\n    return e'

In [33]:
"""Does not work
def fatal_cleaning(e):
    if e != "N" or e != "Y":
        e = "Not specified"
    return e"""

'Does not work\ndef fatal_cleaning(e):\n    if e != "N" or e != "Y":\n        e = "Not specified"\n    return e'

In [34]:
#Clean Date column
#apply clean_date function to our Case number column and put the calculated date into date column
data['Date'] = data['Case Number'].apply(clean_Date)

In [35]:
#create a new column to know if the Date did not contain day and/or month
lst=[]
for e in data['Case Number']:
    if re.search("[0-9]{4}.0{2}.0{2}",e) or re.search("[0-9]{4}.[0-9]{2}.0{2}",e):
        lst.append("Yes")
    else:
        lst.append("No")
data['Date_alterated'] = pd.DataFrame(lst)

In [36]:
#rename our columns name by applying remove space function
data.rename(remove_spaces,axis=1, inplace=True)
data.rename(columns={'Fatal (Y/N)': 'Fatal'}, inplace=True)

In [37]:
#clean country
#Convert Country data into str because some are floats
#I remove "?", considering the country is correct
data['Country'] = data['Country'].astype(str)
data['Country'] = data['Country'].replace('\?', '')
data['Country'] = data['Country'].apply(remove_spaces)

In [38]:
#clean area
data['Area'] = data['Area'].astype(str)
data['Area'] = data['Area'].replace('\"','')
data['Area'] = data['Area'].replace("Tavenui","Taveuni")
data['Area'] = data['Area'].replace("Tamilnadu","Tamil Nadu")
data['Area'] = data['Area'].replace("Guerro","Guerrero")
data['Area'] = data['Area'].replace("Guerrrero","Guerrero")
data['Area'] = data['Area'].replace("islans","Islands")
data['Area'] = data['Area'].replace("Province","")
data['Area'] = data['Area'].replace("Region","")
data['Area'] = data['Area'].replace("shire","")
#data['Area'] = data['Area'].apply(empty_values)
data['Area'] = data['Area'].apply(remove_spaces)

In [39]:
#Clean Location column
data['Location'] = data['Location'].astype(str)
data['Location'] = data['Location'].replace("\(","")
data['Location'] = data['Location'].replace("\)","")
#data['Location'] = data['Location'].apply(empty_values)
data['Location'] = data['Location'].apply(remove_spaces)

In [40]:
#Clean Activity column
data['Activity'] = data['Activity'].astype(str)
#find verb in gerund
lst=[]
for e in data['Activity']:
    lst.append(re.findall("[A-Za-z]*-?ing",e))
data['Activity'] = pd.DataFrame(lst) #loss of information once there is more than one activity on a line
data['Activity'] = data['Activity'].str.capitalize()

In [41]:
#Clean Sex column
data['Sex'] = data['Sex'].astype(str)
data['Sex'] = data['Sex'].replace("N","")
data['Sex'] = data['Sex'].str.replace("\.","") #ne fonctionne pas sans .str
data['Sex'] = data['Sex'].replace("lli","")
data['Sex'] = data['Sex'].replace("nan","")

In [42]:
#data['Sex'] = data['Sex'].apply(empty_values)
data['Sex'] = data['Sex'].apply(remove_spaces)

In [43]:
#Does not work
#data['Sex'] = data['Sex'].apply(sex_cleaning)

In [44]:
#I use Name column information to get the sex if precised
data['Name'] = data['Name'].astype(str)
data['Name'] = data['Name'].str.lower()
lst = []
for e in data['Name']:
    if re.search("boy",e) or re.search("male",e) or re.search("m[ae]n",e):
        lst.append("M")
    elif re.search("girl",e) or re.search("female",e) or re.search("wom[ae]n",e):
        lst.append("F")
    else:
        lst.append("")
data['test sex'] = pd.DataFrame(lst)

In [45]:
#I use newly created column test sex information to complete sex column
data.loc[(data['Sex']=="") & (data['test sex']=="M"), 'Sex'] = "M"
data.loc[(data['Sex']=="") & data['test sex']=="F", 'Sex'] = "F"

In [46]:
#Clean Injury column
data['Injury'] = data['Injury'].astype(str)
data['Injury'] = data['Injury'].replace('\"','')
data['Injury'] = data['Injury'].apply(remove_spaces)

In [47]:
#Clean Fatal column
"""data['Fatal'] = data['Fatal'].astype(str)
data['Fatal'] = data['Fatal'].replace("n","N")
#data['Fatal'] = data['Fatal'].replace("F","")
data['Fatal'] = data['Fatal'].replace("#VALUE!","")
data['Fatal'] = data['Fatal'].replace("nan","")
data['Fatal'] = data['Fatal'].apply(remove_spaces)"""
# does not work
#data['Fatal (Y/N)'] = data['Fatal (Y/N)'].apply(fatal_cleaning)

'data[\'Fatal\'] = data[\'Fatal\'].astype(str)\ndata[\'Fatal\'] = data[\'Fatal\'].replace("n","N")\n#data[\'Fatal\'] = data[\'Fatal\'].replace("F","")\ndata[\'Fatal\'] = data[\'Fatal\'].replace("#VALUE!","")\ndata[\'Fatal\'] = data[\'Fatal\'].replace("nan","")\ndata[\'Fatal\'] = data[\'Fatal\'].apply(remove_spaces)'

In [48]:
#dans injury il y a du fatal qui peut compléter la colonne correspondante
#I use Injury column information to get the fatal if precised
data['Injury'] = data['Injury'].astype(str)
data['Injury'] = data['Injury'].str.lower()
lst = []
for e in data['Injury']:
    if re.search("fatal",e) and not re.search("non-",e):
        lst.append("Y")
    else:
        lst.append("N")
data['test fatal'] = pd.DataFrame(lst)

In [55]:
print(set(data['Fatal']))

{'N', 'n', 'nan', 'Y'}


In [52]:
# Detecting incorrect data in Fatal column
cnt=0
for row in data['Fatal']:
    try:
        if row == 'F':
            data.loc[cnt, 'Fatal']=np.nan
    except ValueError:
        pass
    cnt+=1

data['Fatal'] = data['Fatal'].astype(str)
data['Fatal'] = data['Fatal'].apply(remove_spaces)

In [54]:
#I use newly created column test fatal information to complete fatal column
data.loc[(data['Fatal'].isnull()) & (data['test fatal']=="Y"), 'Fatal'] = "Y"
data.loc[(data['Fatal'].isnull()) & (data['test fatal']=="N"), 'Fatal'] = "N"

In [ ]:
#Drop remaining unwanted columns
to_drop=['test fatal','test sex','Name','Case Number.1','Case Number.2']
data.drop(columns=to_drop,inplace=True)

In [ ]:
data.to_csv('cleaned_data_shark.csv', index=False)